# Optimisation de fonctions univariées

Considérons la fonction suivante à optimiser:
$$
f(x) = x^2 + x -2\sqrt{x}
$$

In [ ]:
f(x) = x^2 + x - 2*sqrt(x)

Je viens de créer une superbe fonction! Voyons à quoi elle ressemble.

In [ ]:
using Plots

In [ ]:
xmin = 0.0
xmax = 1.5
plot(f, xmin, xmax)

## Recherche aléatoire

Une approche naïve est de chercher au hasard un meilleur point. La méthode est facile à implémenter, mais elle inefficace.

In [ ]:
# f: fonction à minimiser
# x0: point de départ
# nmax: nombre d'itération.
function randomsearch(f:: Function, x0:: Vector, nmax:: Int64)
    n = length(x0)
    x = copy(x0)
    value = f(x)
    
    for i = 1:nmax
        Δx = rand(1, n)[1,:]
        xcand = x + Δx
        cand = f(xcand)
        if (cand < value)
            x = xcand
            value = cand
        end
    end
    
    return x, value
end

In [ ]:
function f(x::Vector)
    return x[1] < 0 ? NaN : x[1]^2 + x[1] - 2*sqrt(x[1])
end

In [ ]:
x0 = [0.0]
randomsearch(f, x0, 1000)

Visuellement, la solution trouvée n'est pas très bonne et il est nécessaire de raffiner la recherche.

## Optimisation avec la méthode de Fibonacci

Calculons les nombres de Fibonacci.

In [ ]:
N = 50
F = ones(N)

for i = 3:N
    F[i] = F[i-1] + F[i-2]
end

F

In [ ]:
F[length(F)]

Vérifions que le rapport entre les nombres de Fibonacci successifs converge vers le nombre d'or. Notons que Julia dispose de nombreuses constantes préimplémentées, donc le nombre d'or, disponible avec le nom `golden`.

In [ ]:
Base.MathConstants.golden

Nous pouvons simplifier l'écrire en indiquant explicitement au préalable que nous souhaitons utiliser la librairie `Base.MathConstants`, mais il faudra veiller alors à ce que nos noms de variables n'entrent pas en conflit.

In [ ]:
using Base.MathConstants

golden

Calculons le rapport entre les deux derniers nombres générés à partir de la séquence de Fibonacci, et comparons-le au nombre d'or.

In [ ]:
F[N]/F[N-1]-golden

Nous voyons ainsi que pour $N = 50$, numériquement, le rapport coïncide déjà au nombre d'or.

Revenons à notre problème d'optimisation et supposons que nous savons que la solution est dans [0,1].

In [ ]:
xmin = 0
xmax = 1.0

verbose = true

In [ ]:
function fibonacci(g::Function, xmin, xmax, verbose::Bool = false)
    k = 1
    i = 1
    d = xmax - xmin
    xG = xmin+(F[N-2]/F[N])*d
    xD = xmin+(F[N-1]/F[N])*d
    fG = g(xG)
    fD = g(xD)
    ϵ = 0 # ne servira que pour la dernière itération

    if (verbose)
        println("Iteration $k.\nxmin = $xmin, xmax = $xmax")
        println("xG = $xG, fG = $fG")
        println("xD = $xD, fD = $fD")
        println("d = $d")
    end

    while (k < N-2)
        k += 1
        i += 1
        if k == N-3
            # On est à l'avant-dernière itération.
            # Poser ϵ à 0.1 permet d'éviter de mettre le nouveau point au milieu de l'intervalle.
            ϵ = 0.1
        end
        if fG < fD
            xmax = xD
            d = xmax - xmin
            xD = xG
            fD = fG
            xG = xmin+((F[N-k-1]/F[N-k+1])-ϵ)*d
            fG = g(xG)
        elseif fG > fD
            xmin = xG
            d = xmax - xmin
            xG = xD
            fG = fD
            xD = xmin+((F[N-k]/F[N-k+1])+ϵ)*d
            fD = g(xD)
        elseif fG == fD
            k += 1
            println("Coucou ", k, " ", xG, " ", xD)
            xmin = xG
            xmax = xD
            d = xmax - xmin

            xG = xmin+(F[N-k-1]/F[N-k+1])*d
            fG = g(xG)
            xD = xmin+(F[N-k]/F[N-k+1])*d
            fD = g(xD)
        end
        
        if verbose
            println("Iteration $i.\nxmin = $xmin, xmax = $xmax, $k = k")
            println("xG = $xG, fG = $fG")
            println("xD = $xD, fD = $fD")
            println("d = $d")
        end
    end
    
    if fG < fD
        xmax = xD
    end
    if fG > fD
        xmin = xG
    end
    
    println(k, " ", N, " ", fG, " ", fD)
    
    return [xmin, xmax]
end

In [ ]:
methods(fibonacci)

Essayons la méthode d'abord avec $N = 10$ puis $N = 50$.

In [ ]:
N = 10
bounds = fibonacci(f, xmin, xmax, true)

In [ ]:
N = 50
bounds = fibonacci(f, xmin, xmax, true)

In [ ]:
bounds[2]-bounds[1]

On a très rapidement une très bonne approximation du minimum.

## Méthode du nombre d'or

Commençons par une implémentation basique (voir aussi https://en.wikipedia.org/wiki/Golden-section_search):

In [ ]:
function goldensection(f::Function, a, b, tol::Float64 = 1e-6)

    # Nous commençons en nous assurant que a est plus petit ou égal à b
    (a,b) = (min(a,b),max(a,b))
    d = b - a

    k = 1
    i = 1  # index d'itération
    
    c = b - d / golden
    d = a + d / golden

    while (b-a) > tol
        i += 1
        if f(c) < f(d)
            b = d
        else
            a = c
        end

        c = b - (b - a) / golden
        d = a + (b - a) / golden
    end
    
    println("Nombre d'itérations: ", i)

    return (a, b)
end

In [ ]:
(a, b) = goldensection(f, 0.0, 1.0)

In [ ]:
b-a, (b-a)*golden

In [ ]:
goldensection(f, 0.0, 1.0, 1e-8)

Cela fonctionne, mais l'implémentation nécessite deux évaluations de fonctions à chaque itération, alors que la méthode de Fibonacci n'en exigeait qu'une.

Nous allons capitaliser sur l'idée que les points placés à l'intérieur de l'intervalle de recherche doivent être équidistants des extrémités. Plus précisément, considérons l'intervalle $[a,b]$ et $x \in (a,b)$ avec $x-a < b - x$. Nous cherchons $y$ tel que $x-a = b-y$, et par conséquent
$$
y = a+(b-x).
$$
Comme $x-a < b - x$, $a - x + b > x$, de sorte que $y > x$. Autrement dit, $y$ se trouvera à droite de $x$. Toutefois, si $x$ est trop proche de $a$, la décroissance des longueurs d'intevalle sera très longue. Pour contrer ce problème, nous allons imposer que le rapport entre les longueurs entre les bornes et le point à l'intérieur de l'intervalle.
En supposant que $y$ (i.e. $f(y) > f(x)$) devient la nouvelle borne supérieure, nous avons
$$
\frac{y-x}{x-a} = \frac{x-a}{b-x}
$$
En notant $\alpha = x-a$, $\beta = b-x$ et $\gamma = y-x$, la relation devient
$$
\frac{\gamma}{\alpha} = \frac{\alpha}{\beta}.
$$
Si $f(y) < f(x)$, $x$ devient la nouvelle borne inférieure, et nous devons avoir
$$
\frac{\gamma}{\beta-\gamma} = \frac{\alpha}{\beta}.
$$
Comme $y = a+b-x$, $-\alpha + \beta = \gamma$, ou $\gamma = \beta - \alpha$. Les deux équations sont donc équivalentes et donnent
$$
\frac{\beta-\alpha}{\alpha} = \frac{\alpha}{\beta}
$$
ou
$$
\beta(\beta-\alpha) = \alpha^2,
$$
ce qui équivaut à
$$
\beta^2 - \beta\alpha = \alpha^2
$$
soit
$$
\left(\frac{\beta}{\alpha}\right)^2 - \frac{\beta}{\alpha} = 1.
$$
Cette équation a pour solution
$$
\frac{\beta}{\alpha} = \tau.
$$
En redéveloppant $\alpha$ et $\beta$, nous obtenons
$$
b - x = \tau(x - a)
$$
et donc
$$
(1+\tau)x = b + \tau a
$$
ou encore
$$
x = \frac{b + \tau a}{1+\tau} = a + \frac{b - a}{1+\tau}.
$$
À présent,
\begin{align*}
y & = a + b - x \\
& = a + b - a - \frac{b - a}{1+\tau} \\
& = b - \frac{b - a}{1+\tau} \\
& = \frac{b + \tau b - b + a + \tau a - \tau a}{1+\tau} \\
& = a + \frac{\tau}{1+\tau}(b-a).
\end{align*}
Or
$$
\frac{\tau}{1+\tau}
= \frac{1 + \sqrt{5}}{2 + 1 + \sqrt{5}}
= \frac{1-5}{(1 - \sqrt{5})(3+\sqrt{5}}
= \frac{-4}{-2-2\sqrt{5}}
= \frac{2}{1+\sqrt{5}}
= \frac{1}{\tau}.
$$
Dès lors
$$
y = a + \frac{b-a}{\tau}.
$$

In [ ]:
function goldenrevisited(f::Function, a, b, tol::Float64 = 1e-6)
    # Nous commençons en nous assurant que a est plus petit ou égal à b
    (a,b) = (min(a,b),max(a,b))
    
    d = b-a
    
    # Note: la lettre ϕ est habituellement employée pour le nombre d'or
    invτ = 1/golden
    invτplus = 1/(1+golden)
    
    xG = a + invτplus*d
    xD = a + invτ*d
    fG = f(xG)
    fD = f(xD)

    tol *= golden
    
    i = 1
    while (d > tol)
        i += 1
        if fG < fD
            b = xD
            xD = xG
            d = b-a
            xG = a + invτplus*d
            fD = fG
            fG = f(xG)
        else
            a = xG
            xG = xD
            d = b-a
            xD = a + invτ*d
            fG = fD
            fD = f(xD)
        end
    end

    if fG < fD
        b = xD
    else
        a = xG
    end
    
    println("Nombre d'itérations: ", i)

    return (a,b)
end

In [ ]:
(a,b) = goldenrevisited(f, 0.0, 1.0)

In [ ]:
b-a, (b-a)*golden

## Librairie Optim en Julia

Certaines routines d'optimisation sont directement disponibles en Julia, et peuvent être obtenues avec la commande

In [ ]:
using Pkg
# Pkg.add("Optim")

using Optim

La routine de base est `optimize`, prenant comme premier argument la fonction à minimiser, et pour les fonctions univariées, en deuxième et troisième arguments, les bornes inférieure et supérieure initiales de l'intervalle de recherche.

### Méthode du nombre d'or

Le méthode de recherche de la section dorée peut être appelée avec la function `GoldenSection`.

In [ ]:
result = optimize(f, 0, 1, GoldenSection())

In [ ]:
Optim.minimizer(result)

In [ ]:
bounds

## Méthodes utilisant les dérivées.

La dérivée de $f$ est
$$
f'(x) = 2x+1-\frac{1}{\sqrt{x}},
$$
se qui peut se traduire en Julia comme

In [ ]:
df(x) = 2x+1-1.0/sqrt(x)

Posons $f'(x) = 0$, i.e.
$$
\frac{1}{\sqrt{x}} = 2x+1
$$
ou
$$
\frac{1}{x} = 4x^2 + 4x + 1
$$
Nous devons dès lors chercher les racines du polynôme
$$
4x^3 + 4x^2 + x - 1 = 0.
$$
Pas simple! Nous pouvons cependant utiliser la librairie `Roots`.

In [ ]:
Pkg.add("Roots")
using Roots

In [ ]:
h(x) = x*(4x*(x+1)+1)-1

La fonction `fzeros` cherche à déterminer tous les zéros d'une fonction, mais peut être assez lente. Nous chercherons juste un zéro de la fonction, dans l'intervalle $[0,1]$.

In [ ]:
?fzero

In [ ]:
fzero(h, 0, 1)

In [ ]:
fzeros(h, 0, 1)

Nous retrouvons la même solution que précédemment (heureusement!).

### Méthode de la bisection

Nous pouvons le faire explicitement en codant notre fonction de bisection.

In [ ]:
function bisection(f::Function, a::Float64, b::Float64, δ::Float64 = 1e-8)

    k = 1
    if (a > b)
        c = a
        a = b
        b = c
    end

    fa = f(a)
    fb = f(b)
    if fa == 0
        return k, fa, a, a
    elseif fb == 0
        return k, fb, b, b
    end

    if fa*fb > 0
        println("The function must be of opposite signs at the bounds")
        return
    end

    d = b-a
    c = a+d/2
    fc = f(c)

    while (d > δ)
        if (verbose)
            println("$k. a = $a, b = $b, d = $d, c = $c, fc = $fc")
        end
        k += 1
        if (fc == 0)
            a = b = c
            break
        elseif (fc*fa < 0)
            b = c
            fb = fc
        else
            a = c
            fa = fc
        end
        d = b-a
        c = a+d/2
        fc = f(c)
    end            

    return k, fc, a, b
end

In [ ]:
methods(bisection)

In [ ]:
X = bisection(df, 0.0, 1.0)

In [ ]:
X = bisection(df, 0.0, 1.0, 1e-11)

### Méthode de Newton

La dérivée seconde de $f$ est
$$
f''(x) = 2+\frac{1}{2}x^{-\frac{3}{2}}.
$$

In [ ]:
function d2f(x::Float64)
    return 2+x^(-3/2)/2
end

Une implémentation basique de la méthode de Newton suit.

In [ ]:
function Newton(f::Function, df::Function, d2f:: Function, xstart::Float64, δ::Float64 = 1e-8, nmax::Int64 = 100)
    k = 1
    x = xstart
    if (verbose)
        fx = f(x)
        println("$k. x = $x, f(x) = $fx")
    end
    dfx = df(x)
    while (abs(dfx) > δ && k <= nmax)
        k += 1
        dfx = df(x)
        x = x-dfx/d2f(x)
        if (verbose)
            fx = f(x)
            println("$k. x = $x, f(x) = $fx")
        end
    end
end

In [ ]:
verbose = true
Newton(f, df, d2f, 0.1)

In [ ]:
verbose = true
Newton(f, df, d2f, 100.0)

In [ ]:
x0 = 3.0
x1 = x0-df(x0)/d2f(x0)

Nous voyons que nous convergeons plus rapidement vers la solution optimale quand la méthode fonctionne, mais selon le point de départ, elle peut échouer lamentablement.

## Différentiation numérique

Il n'est pas toujours facile de calculer explicitement la dérivée d'une fonction. Il est cependant possible d'exploiter la définition de dérivée afin de l'approximer numériquement. Soit $f$, dérivable en $x$. La dérivée est définie comme
$$
    f'(x) = \lim_{\epsilon \rightarrow 0} \frac{f(x+\epsilon)-f(x)}{\epsilon}
$$
Nous pouvons dès lors approximer la dérivée en choisissant $\epsilon$ assez petit et en calculant par exemple
$$
f'(x) \approx \frac{f(x+\epsilon)-f(x)}{\epsilon}.
$$

In [ ]:
ϵ = 1e-4
dffd(x) = (f(x+ϵ)-f(x))/ϵ

L'application de la méthode de la bisection à cette approximation permet d'obtenir

In [ ]:
fzero(dffd, 0, 1)

Nous ne pouvons choisir $\epsilon$ arbitrairement petit, comme illustré ci-dessous.

In [ ]:
# fd: Finite difference
dffd(x, ϵ) = (f(x+ϵ)-f(x))/ϵ
x = 1.0
errfd(ϵ) = abs(df(x)-dffd(x, ϵ))
plot(errfd, 1e-14,1e-12)

La méthode peut être affinée en utilisant la différence centrale, définie comme
$$
f'(x) \approx \frac{f(x+\epsilon)-f(x-\epsilon)}{2\epsilon}
$$

In [ ]:
dfcd(x, ϵ=1e-6) = (f(x+ϵ)-f(x-ϵ))/(2*ϵ)

In [ ]:
x = 1.0
errcd(ϵ) = abs(df(x)-dfcd(x, ϵ))
plot(errcd, 1e-18,0.1e-12)

In [ ]:
plot(errcd, 1e-18,0.1e-13)

La différence centrale fournit de plus petites erreurs numériques, mais au prix d'une évaluation de fonction supplémentaire.

Les dérivées numériques sont souvent coûteuses à calculer, surtout pour des problèmes multivariés, et nous nous tournerons vers la différentiation automatique.

In [ ]:
Newton(f, dfcd, d2f, 1.1)

In [ ]:
dfhd(x, ϵ=1e-4) = (dfcd(x+ϵ)-dfcd(x))/ϵ

In [ ]:
Newton(f, dfcd, dfhd, 1.1)

In [ ]:
using ForwardDiff

In [ ]:
g2 = x -> ForwardDiff.derivative(f, x)

In [ ]:
Newton(f, g2, dfhd, 1.1)

In [ ]:
errfd(x) = abs(df(x)-g2(x))
plot(errfd, 1,1.1)